In [1]:
#Filtering Low-Quality Cells and Doublet Detection

In [ ]:
import os
import numpy as np
import scanpy as sc
import pandas as pd
import seaborn as sns

# Set working directory
os.chdir("P:/Tolulope/Cellranger Results/ALL")

In [ ]:
import scvi
import scanpy as sc

from matplotlib.pyplot import rc_context
sc.set_figure_params(dpi=100)

import warnings
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

In [ ]:
# Define sample paths
sample_paths = {
    # Young and old data (Sham and Overload)
    'YSham_GFP': 'P:/Tolulope/Cellranger Results/ALL/YM_Sham_GFP/filtered_feature_bc_matrix',
    'YSham_noGFP': 'P:/Tolulope/Cellranger Results/ALL/YM_Sham_noGFP/filtered_feature_bc_matrix',
    'YOV_GFP': 'P:/Tolulope/Cellranger Results/ALL/YM_OV_GFP/filtered_feature_bc_matrix',
    'YOV_noGFP': 'P:/Tolulope/Cellranger Results/ALL/YM_OV_noGFP/filtered_feature_bc_matrix',
    'ASham_GFP': 'P:/Tolulope/Cellranger Results/ALL/AM_Sham_GFP/filtered_feature_bc_matrix',
    'ASham_noGFP': 'P:/Tolulope/Cellranger Results/ALL/AM_Sham_noGFP/filtered_feature_bc_matrix',
    'AOV_GFP': 'P:/Tolulope/Cellranger Results/ALL/AM_OV_GFP/filtered_feature_bc_matrix',
    'AOV_noGFP': 'P:/Tolulope/Cellranger Results/ALL/AM_OV_noGFP/filtered_feature_bc_matrix',
    
    # Satellite cell depleted data and control    
    'V1': 'P:/Tolulope/Cellranger Results/ALL/v1/outs/filtered_feature_bc_matrix',
    'V2': 'P:/Tolulope/Cellranger Results/ALL/v2/outs/filtered_feature_bc_matrix',
    'T1': 'P:/Tolulope/Cellranger Results/ALL/t1/outs/filtered_feature_bc_matrix',
    'T2': 'P:/Tolulope/Cellranger Results/ALL/t2/outs/filtered_feature_bc_matrix'
}


# Load AnnData objects
adata1 = sc.read_10x_mtx(sample_paths['YSham_GFP'])
adata2 = sc.read_10x_mtx(sample_paths['YSham_noGFP'])
adata3 = sc.read_10x_mtx(sample_paths['YOV_GFP'])
adata4 = sc.read_10x_mtx(sample_paths['YOV_noGFP'])
adata5 = sc.read_10x_mtx(sample_paths['ASham_GFP'])
adata6 = sc.read_10x_mtx(sample_paths['ASham_noGFP'])
adata7 = sc.read_10x_mtx(sample_paths['AOV_GFP'])
adata8 = sc.read_10x_mtx(sample_paths['AOV_noGFP'])
adata9 = sc.read_10x_mtx(sample_paths['V1'])
adata10 = sc.read_10x_mtx(sample_paths['V2'])
adata11 = sc.read_10x_mtx(sample_paths['T1'])
adata12 = sc.read_10x_mtx(sample_paths['T2'])

In [ ]:
sc.pp.filter_genes(adata1, min_cells = 10)
sc.pp.highly_variable_genes(adata1, n_top_genes = 3000, subset = True, flavor = 'seurat_v3')

In [ ]:
scvi.model.SCVI.setup_anndata(adata1)
vae = scvi.model.SCVI(adata1)
vae.train()

In [ ]:
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()
solo.predict()

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)

df

In [ ]:
df.groupby('prediction').count()

In [ ]:
df['dif'] = df.doublet - df.singlet
df

In [ ]:
sns.displot(df[df.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
doublets

In [ ]:
# Define sample paths
sample_paths = {
    'YSham_GFP': 'Y:/Tolulope/Cellranger Results/ALL/YM_Sham_GFP/filtered_feature_bc_matrix',
    
}

# Load AnnData objects
adata1 = sc.read_10x_mtx(sample_paths['YSham_GFP'])
adata1.obs

In [ ]:
adata1.obs['doublet'] = adata1.obs.index.isin(doublets.index) 
adata1.obs

In [ ]:
adata1 = adata1[~adata1.obs.doublet]
adata1

In [ ]:
#sample 2
sc.pp.filter_genes(adata2, min_cells = 10)
adata2

In [ ]:
sc.pp.highly_variable_genes(adata2, n_top_genes = 3000, subset = True, flavor = 'seurat_v3')

In [ ]:
scvi.model.SCVI.setup_anndata(adata2)
vae = scvi.model.SCVI(adata2)
vae.train()

In [ ]:
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()
solo.predict()

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)
df

In [ ]:
df.groupby('prediction').count()

In [ ]:
df['dif'] = df.doublet - df.singlet
df

In [ ]:
sns.displot(df[df.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets = df[(df.prediction == 'doublet') & (df.dif > 0.8)]
doublets

In [ ]:
# Define sample paths
sample_paths = {
    'YSham_noGFP':'Y:/Tolulope/Cellranger Results/ALL/YM_Sham_noGFP/filtered_feature_bc_matrix',
    
}

# Load AnnData objects

adata2 = sc.read_10x_mtx(sample_paths['YSham_noGFP'])
adata2.obs

In [ ]:
adata2.obs['doublet'] = adata2.obs.index.isin(doublets.index) 
adata2.obs

In [ ]:
adata2 = adata2[~adata2.obs.doublet]
adata2

In [ ]:
#sample 3
sc.pp.filter_genes(adata3, min_cells = 10)
adata3

In [ ]:
sc.pp.highly_variable_genes(adata3, n_top_genes = 3000, subset = True, flavor = 'seurat_v3')

In [ ]:
scvi.model.SCVI.setup_anndata(adata3)
vae = scvi.model.SCVI(adata3)
vae.train()


In [ ]:
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()
solo.predict()

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)
df

In [ ]:
df.groupby('prediction').count()

In [ ]:
df['dif'] = df.doublet - df.singlet
df

In [ ]:
sns.displot(df[df.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
doublets

In [ ]:
# Define sample paths
sample_paths = {
    'YOV_GFP': 'Y:/Tolulope/Cellranger Results/ALL/YM_OV_GFP/filtered_feature_bc_matrix',
    
}

# Load AnnData objects

adata3 = sc.read_10x_mtx(sample_paths['YOV_GFP'])
adata3.obs

In [ ]:
adata3.obs['doublet'] = adata3.obs.index.isin(doublets.index) 
adata3.obs

In [ ]:
adata3 = adata3[~adata3.obs.doublet]
adata3

In [ ]:
#sample 4
sc.pp.filter_genes(adata4, min_cells = 10)
adata4

In [ ]:
sc.pp.highly_variable_genes(adata4, n_top_genes = 3000, subset = True, flavor = 'seurat_v3')

In [ ]:
scvi.model.SCVI.setup_anndata(adata4)
vae = scvi.model.SCVI(adata4)
vae.train()

In [ ]:
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()
solo.predict()

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)
df

In [ ]:
df.groupby('prediction').count()

In [ ]:
df['dif'] = df.doublet - df.singlet
df

In [ ]:
sns.displot(df[df.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
doublets

In [ ]:
# Define sample paths
sample_paths = {
    'YOV_noGFP': 'Y:/Tolulope/Cellranger Results/ALL/YM_OV_noGFP/filtered_feature_bc_matrix',
    
}

# Load AnnData objects

adata4 = sc.read_10x_mtx(sample_paths['YOV_noGFP'])
adata4.obs

In [ ]:
adata4.obs['doublet'] = adata4.obs.index.isin(doublets.index) 
adata4.obs

In [ ]:
adata4 = adata4[~adata4.obs.doublet]
adata4

In [ ]:
#SAMPLE5
sc.pp.filter_genes(adata5, min_cells = 10)
sc.pp.highly_variable_genes(adata5, n_top_genes = 3000, subset = True, flavor = 'seurat_v3')

In [ ]:
scvi.model.SCVI.setup_anndata(adata5)
vae = scvi.model.SCVI(adata5)
vae.train()


In [ ]:
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()
solo.predict()

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)

df

In [ ]:
df.groupby('prediction').count()

In [ ]:
df['dif'] = df.doublet - df.singlet
df

In [ ]:
sns.displot(df[df.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets = df[(df.prediction == 'doublet') & (df.dif > 0.8)]
doublets

In [ ]:
# Define sample paths
sample_paths = {
    'ASham_GFP': 'Y:/Tolulope/Cellranger Results/ALL/AM_Sham_GFP/filtered_feature_bc_matrix',
    
}

# Load AnnData objects

adata5 = sc.read_10x_mtx(sample_paths['ASham_GFP'])
adata5.obs

In [ ]:
adata5.obs['doublet'] = adata5.obs.index.isin(doublets.index) 
adata5.obs

In [ ]:
adata5 = adata5[~adata5.obs.doublet]
adata5

In [ ]:
#sample 6
sc.pp.filter_genes(adata6, min_cells = 10)
sc.pp.highly_variable_genes(adata6, n_top_genes = 3000, subset = True, flavor = 'seurat_v3')

In [ ]:
scvi.model.SCVI.setup_anndata(adata6)
vae = scvi.model.SCVI(adata6)
vae.train()


In [ ]:
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()
solo.predict()

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)

df

In [ ]:
df.groupby('prediction').count()

In [ ]:
df['dif'] = df.doublet - df.singlet
df

In [ ]:
sns.displot(df[df.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets = df[(df.prediction == 'doublet') & (df.dif > 1.2)]
doublets

In [ ]:
# Define sample paths
sample_paths = {
    
    'ASham_noGFP':'Y:/Tolulope/Cellranger Results/ALL/AM_Sham_noGFP/filtered_feature_bc_matrix',
    
}

# Load AnnData objects

adata6 = sc.read_10x_mtx(sample_paths['ASham_noGFP'])
adata6.obs

In [ ]:
adata6.obs['doublet'] = adata6.obs.index.isin(doublets.index) 
adata6.obs

In [ ]:
adata6 = adata6[~adata6.obs.doublet]
adata6

In [ ]:
#sample 7
sc.pp.filter_genes(adata7, min_cells = 10)
sc.pp.highly_variable_genes(adata7, n_top_genes = 3000, subset = True, flavor = 'seurat_v3')

In [ ]:
scvi.model.SCVI.setup_anndata(adata7)
vae = scvi.model.SCVI(adata7)
vae.train()


In [ ]:
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()
solo.predict()

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)

df

In [ ]:
df.groupby('prediction').count()

In [ ]:
df['dif'] = df.doublet - df.singlet
df

In [ ]:
sns.displot(df[df.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
doublets

In [ ]:
# Define sample paths
sample_paths = {
    'AOV_GFP': 'Y:/Tolulope/Cellranger Results/ALL/AM_OV_GFP/filtered_feature_bc_matrix',
    
}

# Load AnnData objects

adata7 = sc.read_10x_mtx(sample_paths['AOV_GFP'])
adata7.obs

In [ ]:
adata7.obs['doublet'] = adata7.obs.index.isin(doublets.index) 
adata7.obs

In [ ]:
adata7 = adata7[~adata7.obs.doublet]
adata7

In [ ]:
#sample8
sc.pp.filter_genes(adata8, min_cells = 10)
sc.pp.highly_variable_genes(adata8, n_top_genes = 3000, subset = True, flavor = 'seurat_v3')

In [ ]:
scvi.model.SCVI.setup_anndata(adata8)
vae = scvi.model.SCVI(adata8)
vae.train()


In [ ]:
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()
solo.predict()

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)

df

In [ ]:
df.groupby('prediction').count()

In [ ]:
df['dif'] = df.doublet - df.singlet
df

In [ ]:
sns.displot(df[df.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
doublets

In [ ]:
# Define sample paths
sample_paths = {
    'AOV_noGFP': 'Y:/Tolulope/Cellranger Results/ALL/AM_OV_noGFP/filtered_feature_bc_matrix',
    
}

# Load AnnData objects

adata8 = sc.read_10x_mtx(sample_paths['AOV_noGFP'])
adata8.obs

In [ ]:
adata8.obs['doublet'] = adata8.obs.index.isin(doublets.index) 
adata8.obs

In [ ]:
adata8 = adata8[~adata8.obs.doublet]
adata8

In [ ]:
#sample 9
sc.pp.filter_genes(adata9, min_cells = 10)
sc.pp.highly_variable_genes(adata9, n_top_genes = 3000, subset = True, flavor = 'seurat_v3')

In [ ]:
scvi.model.SCVI.setup_anndata(adata9)
vae = scvi.model.SCVI(adata9)
vae.train()


In [ ]:
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()
solo.predict()

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)
df

In [ ]:
df.groupby('prediction').count()

In [ ]:
df['dif'] = df.doublet - df.singlet
df

In [ ]:
sns.displot(df[df.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
doublets

In [ ]:
# Define sample paths
sample_paths = {
    'V1': 'Y:/Tolulope/Cellranger Results/ALL/v1/outs/filtered_feature_bc_matrix',
    
}

# Load AnnData objects

adata9 = sc.read_10x_mtx(sample_paths['V1'])
adata9.obs

In [ ]:
adata9.obs['doublet'] = adata9.obs.index.isin(doublets.index) 
adata9.obs

In [ ]:
adata9 = adata9[~adata9.obs.doublet]
adata9

In [ ]:
#sample10
sc.pp.filter_genes(adata10, min_cells = 10)
sc.pp.highly_variable_genes(adata10, n_top_genes = 3000, subset = True, flavor = 'seurat_v3')

In [ ]:
scvi.model.SCVI.setup_anndata(adata10)
vae = scvi.model.SCVI(adata10)
vae.train()

In [ ]:
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()
solo.predict()

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)
df

In [ ]:
df.groupby('prediction').count()

In [ ]:
df['dif'] = df.doublet - df.singlet
df

In [ ]:
sns.displot(df[df.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
doublets

In [ ]:
# Define sample paths
sample_paths = {
    'V2': 'Y:/Tolulope/Cellranger Results/ALL/v2/outs/filtered_feature_bc_matrix',
    
}

# Load AnnData objects

adata10 = sc.read_10x_mtx(sample_paths['V2'])
adata10.obs

In [ ]:
adata10.obs['doublet'] = adata10.obs.index.isin(doublets.index) 
adata10.obs

In [ ]:
adata10 = adata10[~adata10.obs.doublet]
adata10

In [ ]:
#sample11
sc.pp.filter_genes(adata11, min_cells = 10)
sc.pp.highly_variable_genes(adata11, n_top_genes = 3000, subset = True, flavor = 'seurat_v3')

In [ ]:
scvi.model.SCVI.setup_anndata(adata11)
vae = scvi.model.SCVI(adata11)
vae.train()

In [ ]:
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()
solo.predict()

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)
df

In [ ]:
df.groupby('prediction').count()

In [ ]:
df['dif'] = df.doublet - df.singlet
df

In [ ]:
sns.displot(df[df.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
doublets

In [ ]:
# Define sample paths
sample_paths = {
    'T1': 'Y:/Tolulope/Cellranger Results/ALL/t1/outs/filtered_feature_bc_matrix',
   
}

# Load AnnData objects

adata11 = sc.read_10x_mtx(sample_paths['T1'])
adata11.obs

In [ ]:
adata11.obs['doublet'] = adata11.obs.index.isin(doublets.index) 
adata11.obs

In [ ]:
adata11 = adata11[~adata11.obs.doublet]
adata11

In [ ]:
#sample 12
sc.pp.filter_genes(adata12, min_cells = 10)
sc.pp.highly_variable_genes(adata12, n_top_genes = 3000, subset = True, flavor = 'seurat_v3')

In [ ]:
scvi.model.SCVI.setup_anndata(adata12)
vae = scvi.model.SCVI(adata12)
vae.train()

In [ ]:
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()
solo.predict()

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)
df

In [ ]:
df.groupby('prediction').count()

In [ ]:
df['dif'] = df.doublet - df.singlet
df

In [ ]:
sns.displot(df[df.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets = df[(df.prediction == 'doublet') & (df.dif > 0.8)]
doublets

In [ ]:
# Define sample paths
sample_paths = {
    'T2': 'Y:/Tolulope/Cellranger Results/ALL/t2/outs/filtered_feature_bc_matrix'
}

# Load AnnData objects

adata12 = sc.read_10x_mtx(sample_paths['T2'])
adata12.obs

In [ ]:
adata12.obs['doublet'] = adata12.obs.index.isin(doublets.index) 
adata12.obs

In [ ]:
adata12 = adata12[~adata12.obs.doublet]
adata12

In [ ]:
# Make variable names unique
for adata in [adata1, adata2, adata3, adata4, adata5, adata6, adata7, adata8, adata9, adata10, adata11, adata12]:
    adata.var_names_make_unique()
    adata.raw = adata

In [ ]:
# Quality Control (QC)
for adata in [adata1, adata2, adata3, adata4, adata5, adata6, adata7, adata8, adata9, adata10, adata11, adata12]:
    sc.pp.filter_cells(adata, min_genes=200)#get rid of cells with fewer than 200 genes

In [ ]:
# Annotate mitochondrial genes
for adata in [adata1, adata2, adata3, adata4, adata5, adata6, adata7, adata8, adata9, adata10, adata11, adata12]:
    adata.var['mt'] = adata.var_names.str.startswith('mt-')  # Assuming mitochondrial genes start with 'MT-'
adata1.var[adata4.var.mt == True]

In [ ]:
# Annotate ribosomal genes
for adata in [adata1, adata2, adata3, adata4, adata5, adata6, adata7, adata8, adata9, adata10, adata11, adata12]:
    adata.var['ribo'] = adata.var_names.str.startswith(('Rps', 'Rpl'))
adata1.var[adata4.var['ribo']]  

In [ ]:
# Annotate ribosomal genes
for adata in [adata1, adata2, adata3, adata4, adata5, adata6, adata7, adata8, adata9, adata10, adata11, adata12]:
    adata.var['hb'] = adata.var_names.str.contains('^Hb[^(P)]', regex=True)
adata4.var[adata4.var['hb']]

In [ ]:
#SAMPLE1
sc.pp.calculate_qc_metrics(adata1, qc_vars=['mt', 'ribo', 'hb'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata1, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo','pct_counts_hb'],
             jitter=0.4, multi_panel=True)

In [ ]:
#instead of picking subjectively, you can use quanitle
upper_lim = np.quantile(adata1.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata1.obs.n_genes_by_counts.values, .02)
print(f'{lower_lim} to {upper_lim}')

In [ ]:
adata1 = adata1[(adata1.obs.n_genes_by_counts < upper_lim) & (adata1.obs.n_genes_by_counts > lower_lim)]
adata1 = adata1[adata1.obs.pct_counts_mt < 15]
adata1 = adata1[adata1.obs.pct_counts_ribo < 40]
adata1 = adata1[adata1.obs['pct_counts_hb'] < 0.1].copy()

sc.pl.violin(adata1, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, multi_panel=True)
adata1

In [ ]:
#SAMPLE2
sc.pp.calculate_qc_metrics(adata2, qc_vars=['mt', 'ribo', 'hb'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata2, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo','pct_counts_hb'],
             jitter=0.4, multi_panel=True)

In [ ]:
#instead of picking subjectively, you can use quanitle
upper_lim = np.quantile(adata2.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata2.obs.n_genes_by_counts.values, .02)
print(f'{lower_lim} to {upper_lim}')

In [ ]:
adata2 = adata2[(adata2.obs.n_genes_by_counts < upper_lim) & (adata2.obs.n_genes_by_counts > lower_lim)]
adata2 = adata2[adata2.obs.pct_counts_mt < 10]
adata2 = adata2[adata2.obs.pct_counts_ribo < 25]
adata2 = adata2[adata2.obs['pct_counts_hb'] < 0.1].copy()

sc.pl.violin(adata2, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, multi_panel=True)
adata2

In [ ]:
#SAMPLE3
sc.pp.calculate_qc_metrics(adata3, qc_vars=['mt', 'ribo', 'hb'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata3, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo','pct_counts_hb'],
             jitter=0.4, multi_panel=True)

In [ ]:
#instead of picking subjectively, you can use quanitle
upper_lim = np.quantile(adata3.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata3.obs.n_genes_by_counts.values, .02)
print(f'{lower_lim} to {upper_lim}')

In [ ]:
adata3 = adata3[(adata3.obs.n_genes_by_counts < upper_lim) & (adata3.obs.n_genes_by_counts > lower_lim)]
adata3 = adata3[adata3.obs.pct_counts_mt < 20]
adata3 = adata3[adata3.obs.pct_counts_ribo < 40]
adata3 = adata3[adata3.obs['pct_counts_hb'] < 0.1].copy()

sc.pl.violin(adata3, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, multi_panel=True)
adata3

In [ ]:
#SAMPLE4
sc.pp.calculate_qc_metrics(adata4, qc_vars=['mt', 'ribo', 'hb'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata4, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo','pct_counts_hb'],
             jitter=0.4, multi_panel=True)

In [ ]:
#instead of picking subjectively, you can use quanitle
upper_lim = np.quantile(adata4.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata4.obs.n_genes_by_counts.values, .02)
print(f'{lower_lim} to {upper_lim}')

In [ ]:
adata4 = adata4[(adata4.obs.n_genes_by_counts < upper_lim) & (adata4.obs.n_genes_by_counts > lower_lim)]
adata4 = adata4[adata4.obs.pct_counts_mt < 15]
adata4 = adata4[adata4.obs.pct_counts_ribo < 30]
adata4 = adata4[adata4.obs['pct_counts_hb'] < 0.1].copy()

sc.pl.violin(adata4, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, multi_panel=True)
adata4

In [ ]:
#SAMPLE5
sc.pp.calculate_qc_metrics(adata5, qc_vars=['mt', 'ribo', 'hb'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata5, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo','pct_counts_hb'],
             jitter=0.4, multi_panel=True)

In [ ]:
#instead of picking subjectively, you can use quanitle
upper_lim = np.quantile(adata5.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata5.obs.n_genes_by_counts.values, .02)
print(f'{lower_lim} to {upper_lim}')

In [ ]:
adata5 = adata5[(adata5.obs.n_genes_by_counts < upper_lim) & (adata5.obs.n_genes_by_counts > lower_lim)]
adata5 = adata5[adata5.obs.pct_counts_mt < 20]
adata5= adata5[adata5.obs.pct_counts_ribo < 40]
adata5 = adata5[adata5.obs['pct_counts_hb'] < 0.1].copy()

sc.pl.violin(adata5, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, multi_panel=True)
adata5

In [ ]:
#SAMPLE6
sc.pp.calculate_qc_metrics(adata6, qc_vars=['mt', 'ribo', 'hb'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata6, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo','pct_counts_hb'],
             jitter=0.4, multi_panel=True)

In [ ]:
#instead of picking subjectively, you can use quanitle
upper_lim = np.quantile(adata6.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata6.obs.n_genes_by_counts.values, .02)
print(f'{lower_lim} to {upper_lim}')

In [ ]:
adata6 = adata6[(adata6.obs.n_genes_by_counts < upper_lim) & (adata6.obs.n_genes_by_counts > lower_lim)]
adata6 = adata6[adata6.obs.pct_counts_mt < 20]
adata6 = adata6[adata6.obs.pct_counts_ribo < 28]
adata6 = adata6[adata6.obs['pct_counts_hb'] < 0.1].copy()

sc.pl.violin(adata6, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, multi_panel=True)
adata6

In [ ]:
#SAMPLE7
sc.pp.calculate_qc_metrics(adata7, qc_vars=['mt', 'ribo', 'hb'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata7, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo','pct_counts_hb'],
             jitter=0.4, multi_panel=True)

In [ ]:
#instead of picking subjectively, you can use quanitle
upper_lim = np.quantile(adata7.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata7.obs.n_genes_by_counts.values, .02)
print(f'{lower_lim} to {upper_lim}')

In [ ]:
adata7 = adata7[(adata7.obs.n_genes_by_counts < upper_lim) & (adata7.obs.n_genes_by_counts > lower_lim)]
adata7 = adata7[adata7.obs.pct_counts_mt < 20]
adata7 = adata7[adata7.obs.pct_counts_ribo < 25]
adata7 = adata7[adata7.obs['pct_counts_hb'] < 0.1].copy()

sc.pl.violin(adata7, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, multi_panel=True)
adata7

In [ ]:
#SAMPLE8
sc.pp.calculate_qc_metrics(adata8, qc_vars=['mt', 'ribo', 'hb'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata8, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo','pct_counts_hb'],
             jitter=0.4, multi_panel=True)

In [ ]:
#instead of picking subjectively, you can use quanitle
upper_lim = np.quantile(adata8.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata8.obs.n_genes_by_counts.values, .02)
print(f'{lower_lim} to {upper_lim}')

In [ ]:
adata8 = adata8[(adata8.obs.n_genes_by_counts < upper_lim) & (adata8.obs.n_genes_by_counts > lower_lim)]
adata8 = adata8[adata8.obs.pct_counts_mt < 20]
adata8 = adata8[adata8.obs.pct_counts_ribo < 25]
adata8 = adata8[adata8.obs['pct_counts_hb'] < 0.1].copy()

sc.pl.violin(adata8, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, multi_panel=True)
adata8

In [ ]:
#SAMPLE9
sc.pp.calculate_qc_metrics(adata9, qc_vars=['mt', 'ribo', 'hb'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata9, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo','pct_counts_hb'],
             jitter=0.4, multi_panel=True)

In [ ]:
#instead of picking subjectively, you can use quanitle
upper_lim = np.quantile(adata9.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata9.obs.n_genes_by_counts.values, .02)
print(f'{lower_lim} to {upper_lim}')

In [ ]:
adata9 = adata9[(adata9.obs.n_genes_by_counts < upper_lim) & (adata9.obs.n_genes_by_counts > lower_lim)]
adata9 = adata9[adata9.obs.pct_counts_mt < 10]
adata9 = adata9[adata9.obs.pct_counts_ribo < 40]
adata9 = adata9[adata9.obs['pct_counts_hb'] < 0.1].copy()

sc.pl.violin(adata9, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, multi_panel=True)
adata9

In [ ]:
#SAMPLE10
sc.pp.calculate_qc_metrics(adata10, qc_vars=['mt', 'ribo', 'hb'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata10, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo','pct_counts_hb'],
             jitter=0.4, multi_panel=True)

In [ ]:
#instead of picking subjectively, you can use quanitle
upper_lim = np.quantile(adata10.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata10.obs.n_genes_by_counts.values, .02)
print(f'{lower_lim} to {upper_lim}')

In [ ]:
adata10 = adata10[(adata10.obs.n_genes_by_counts < upper_lim) & (adata10.obs.n_genes_by_counts > lower_lim)]
adata10 = adata10[adata10.obs.pct_counts_mt < 10]
adata10 = adata10[adata10.obs.pct_counts_ribo < 40]
adata10 = adata10[adata10.obs['pct_counts_hb'] < 0.1].copy()

sc.pl.violin(adata10, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, multi_panel=True)
adata10

In [ ]:
#SAMPLE11
sc.pp.calculate_qc_metrics(adata11, qc_vars=['mt', 'ribo', 'hb'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata11, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo','pct_counts_hb'],
             jitter=0.4, multi_panel=True)

In [ ]:
#instead of picking subjectively, you can use quanitle
upper_lim = np.quantile(adata11.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata11.obs.n_genes_by_counts.values, .02)
print(f'{lower_lim} to {upper_lim}')

In [ ]:
adata11 = adata11[(adata11.obs.n_genes_by_counts < upper_lim) & (adata11.obs.n_genes_by_counts > lower_lim)]
adata11 = adata11[adata11.obs.pct_counts_mt < 10]
adata11 = adata11[adata11.obs.pct_counts_ribo < 40]
adata11 = adata11[adata11.obs['pct_counts_hb'] < 0.1].copy()

sc.pl.violin(adata11, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, multi_panel=True)
adata11

In [ ]:
#SAMPLE12
sc.pp.calculate_qc_metrics(adata12, qc_vars=['mt', 'ribo', 'hb'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata12, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo','pct_counts_hb'],
             jitter=0.4, multi_panel=True)

In [ ]:
#instead of picking subjectively, you can use quanitle
upper_lim = np.quantile(adata12.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata12.obs.n_genes_by_counts.values, .02)
print(f'{lower_lim} to {upper_lim}')

In [ ]:
adata12 = adata12[(adata12.obs.n_genes_by_counts < upper_lim) & (adata12.obs.n_genes_by_counts > lower_lim)]
adata12 = adata12[adata12.obs.pct_counts_mt < 10]
adata12 = adata12[adata12.obs.pct_counts_ribo < 35]
adata12 = adata12[adata12.obs['pct_counts_hb'] < 0.1].copy()

sc.pl.violin(adata12, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, multi_panel=True)
adata12

In [ ]:
#combine young and old data

In [ ]:
import scanpy as sc

# Assuming adatas is a list containing your individual AnnData objects: [adata1, adata2, adata3, adata4]
adatayoungold = [adata1, adata2, adata3, adata4, adata5, adata6, adata7, adata8]

# Define sample names
sample_names = ['YSham_GFP', 'YSham_noGFP', 'YOV_GFP', 'YOV_noGFP', 'ASham_GFP', 'ASham_noGFP', 'AOV_GFP', 'AOV_noGFP']


# Concatenate the list of AnnData objects into a single AnnData object
adataa = sc.concat(adatayoungold, join='outer', label='Sample', keys=sample_names)
adataa

In [ ]:
sc.pp.filter_genes(adataa, min_cells = 10)
adataa

In [ ]:
adataa.X

In [ ]:
adataa.write_h5ad('combined1.h5ad')

In [ ]:
# combine satellite cell depleted data and control

In [ ]:
import scanpy as sc

# Assuming adatas is a list containing your individual AnnData objects: [adata1, adata2, adata3, adata4]
adatavehtam = [adata9, adata10, adata11, adata12]

# Define sample names
sample_names = ['V1', 'V2', 'T1', 'T2']


# Concatenate the list of AnnData objects into a single AnnData object
adatadata = sc.concat(adatavehtam, join='outer', label='Sample', keys=sample_names)


In [ ]:
adatadata

In [ ]:
sc.pp.filter_genes(adatadata, min_cells = 10)
adatadata

In [ ]:
adatadata.X

In [ ]:
adatadata.write_h5ad('combined2.h5ad')